In [1]:
# Import libraries and set desired options
%matplotlib inline
from matplotlib import pyplot as plt

import pickle
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

Notebook by Yuri Kashnitsky, edited by Ivan Komarov. 

In this competition we are going to analyze a sequence of websites visited by a person to predict whether this person is Alice or not. The metric of evaluation is [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). 

###  Data Downloading and Transformation
First, read the training and test sets. 

In [2]:
times = ['time'+str(i) for i in range(1,11)]
times

['time1',
 'time2',
 'time3',
 'time4',
 'time5',
 'time6',
 'time7',
 'time8',
 'time9',
 'time10']

In [3]:
# Read the training and test data sets and parse dates
train_df = pd.read_csv('train.csv',
                       index_col='session_id', parse_dates=times)

test_df = pd.read_csv('test.csv',
                      index_col='session_id', parse_dates=times)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
27554,41475,2013-11-15 07:39:35,6725.0,2013-11-15 07:39:35,6725.0,2013-11-15 07:39:36,41475.0,2013-11-15 07:39:36,41476.0,2013-11-15 07:39:40,...,2013-11-15 07:39:41,6725.0,2013-11-15 07:42:50,41475.0,2013-11-15 07:42:50,41476.0,2013-11-15 07:42:50,6725.0,2013-11-15 07:44:25,0
81350,41476,2013-11-15 07:44:25,41475.0,2013-11-15 07:44:25,41476.0,2013-11-15 07:57:45,6725.0,2013-11-15 07:57:45,41475.0,2013-11-15 07:57:45,...,2013-11-15 07:57:46,41476.0,2013-11-15 07:57:47,6725.0,2013-11-15 07:57:49,41475.0,2013-11-15 07:57:49,41476.0,2013-11-15 07:57:49,0
234665,4802,2013-11-15 07:52:17,23.0,2013-11-15 07:52:18,4803.0,2013-11-15 07:52:19,38.0,2013-11-15 07:52:19,38.0,2013-11-15 07:52:20,...,2013-11-15 07:52:20,4804.0,2013-11-15 07:52:23,21.0,2013-11-15 07:52:26,23.0,2013-11-15 07:52:26,22.0,2013-11-15 07:52:28,0
97610,23,2013-11-15 07:52:28,23.0,2013-11-15 07:52:29,22.0,2013-11-15 07:52:37,21.0,2013-11-15 07:52:37,63.0,2013-11-15 07:55:10,...,2013-11-15 07:55:10,784.0,2013-11-15 07:55:56,4804.0,2013-11-15 07:57:50,4804.0,2013-11-15 08:01:18,784.0,2013-11-15 08:01:26,0
161358,41476,2013-11-15 07:57:50,41476.0,2013-11-15 07:57:51,6725.0,2013-11-15 07:59:34,41475.0,2013-11-15 07:59:34,41476.0,2013-11-15 07:59:34,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 27554 to 11690
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   250098 non-null  float64       
 3   time2   250098 non-null  datetime64[ns]
 4   site3   246919 non-null  float64       
 5   time3   246919 non-null  datetime64[ns]
 6   site4   244321 non-null  float64       
 7   time4   244321 non-null  datetime64[ns]
 8   site5   241829 non-null  float64       
 9   time5   241829 non-null  datetime64[ns]
 10  site6   239495 non-null  float64       
 11  time6   239495 non-null  datetime64[ns]
 12  site7   237297 non-null  float64       
 13  time7   237297 non-null  datetime64[ns]
 14  site8   235224 non-null  float64       
 15  time8   235224 non-null  datetime64[ns]
 16  site9   233084 non-null  float64       
 17  time9   233084 non-null  d

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   site1   82797 non-null  int64         
 1   time1   82797 non-null  datetime64[ns]
 2   site2   81308 non-null  float64       
 3   time2   81308 non-null  datetime64[ns]
 4   site3   80075 non-null  float64       
 5   time3   80075 non-null  datetime64[ns]
 6   site4   79182 non-null  float64       
 7   time4   79182 non-null  datetime64[ns]
 8   site5   78341 non-null  float64       
 9   time5   78341 non-null  datetime64[ns]
 10  site6   77566 non-null  float64       
 11  time6   77566 non-null  datetime64[ns]
 12  site7   76840 non-null  float64       
 13  time7   76840 non-null  datetime64[ns]
 14  site8   76151 non-null  float64       
 15  time8   76151 non-null  datetime64[ns]
 16  site9   75484 non-null  float64       
 17  time9   75484 non-null  datetime64[ns]
 18  site10

The training data set contains the following features:

- **site1** – ID of the first visited website in the session
- **time1** – visiting time for the first website in the session
- ...
- **site10** – ID of the tenth visited website in the session
- **time10** – visiting time for the tenth website in the session
- **target** – target variable, equals 1 for Alice's sessions, and 0 otherwise
    
**User sessions end either if a user has visited ten websites or if a session has lasted over thirty minutes.**

There are some empty values in the table, it means that some sessions contain less than ten websites. Replace empty values with 0 and change columns types to integer. Also load the websites dictionary and check how it looks:

In [6]:
# Change site1, ..., site10 columns type to integer and fill NA-values with zeros
sites = ['site'+str(i) for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# Load websites dictionary
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)
    
# r before a string means "raw", i.e. take the string as it comes,
# e.g. as a file path without interpreting special symbols like \n

print('Websites total:', len(site_dict))



Websites total: 48371


In [7]:
# See what's in the dict
list(site_dict.items())[:3]

[('www.abmecatronique.com', 25075),
 ('groups.live.com', 13997),
 ('majeureliguefootball.wordpress.com', 42436)]

In [8]:
# Size of the sets
print(test_df.shape, train_df.shape)

(82797, 20) (253561, 21)


In [9]:
train_df_times = train_df[times].fillna(0)

train_df_times.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 27554 to 11690
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   time1   253561 non-null  datetime64[ns]
 1   time2   253561 non-null  object        
 2   time3   253561 non-null  object        
 3   time4   253561 non-null  object        
 4   time5   253561 non-null  object        
 5   time6   253561 non-null  object        
 6   time7   253561 non-null  object        
 7   time8   253561 non-null  object        
 8   time9   253561 non-null  object        
 9   time10  253561 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 21.3+ MB


In [10]:
test_df_times = test_df[times].fillna(0)
test_df_times.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time1   82797 non-null  datetime64[ns]
 1   time2   82797 non-null  object        
 2   time3   82797 non-null  object        
 3   time4   82797 non-null  object        
 4   time5   82797 non-null  object        
 5   time6   82797 non-null  object        
 6   time7   82797 non-null  object        
 7   time8   82797 non-null  object        
 8   time9   82797 non-null  object        
 9   time10  82797 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 6.9+ MB


In [11]:
train_df_times['first_visit'] = train_df['time1'].dt.time
train_df_times['month'] = train_df['time1'].dt.month_name()
train_df_times['day_name'] = train_df['time1'].dt.day_name()
train_df_times['year'] = train_df['time1'].dt.year
for i in range(1, 10):
    train_df_times[f'seconds_{i}'] = (train_df[f'time{i+1}'] - train_df[f'time{i}']).dt.components.seconds
seconds = ['seconds_'+str(i) for i in range(1, 10)]
train_df_times['sum_seconds'] = train_df_times[seconds].sum(axis=1)
train_df_times['mean_seconds'] = train_df_times[seconds].mean(axis=1)
train_df_times = train_df_times.drop(times, axis=1).fillna(0)
train_df_times

,first_visit,month,day_name,year,seconds_1,seconds_2,seconds_3,seconds_4,seconds_5,seconds_6,seconds_7,seconds_8,seconds_9,sum_seconds,mean_seconds
session_id,,,,,,,,,,,,,,,
27554,07:39:35,November,Friday,2013,0.0,1.0,0.0,4.0,1.0,9.0,0.0,0.0,35.0,50.0,5.555556
81350,07:44:25,November,Friday,2013,0.0,20.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,24.0,2.666667
234665,07:52:17,November,Friday,2013,1.0,1.0,0.0,1.0,0.0,3.0,3.0,0.0,2.0,11.0,1.222222
97610,07:52:28,November,Friday,2013,1.0,8.0,0.0,33.0,0.0,46.0,54.0,28.0,8.0,178.0,19.777778
161358,07:57:50,November,Friday,2013,1.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,11.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129606,18:06:13,May,Saturday,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
53628,18:09:26,May,Saturday,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
84754,18:11:21,May,Saturday,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [12]:
test_df_times['first_visit'] = test_df['time1'].dt.time
test_df_times['month'] = test_df['time1'].dt.month_name()
test_df_times['day_name'] = test_df['time1'].dt.day_name()
test_df_times['year'] = test_df['time1'].dt.year
for i in range(1, 10):
    test_df_times[f'seconds_{i}'] = (test_df[f'time{i+1}'] - test_df[f'time{i}']).dt.components.seconds
seconds = ['seconds_'+str(i) for i in range(1, 10)]
test_df_times['sum_seconds'] = test_df_times[seconds].sum(axis=1)
test_df_times['mean_seconds'] = test_df_times[seconds].mean(axis=1)
test_df_times = test_df_times.drop(times, axis=1).fillna(0)
test_df_times

,first_visit,month,day_name,year,seconds_1,seconds_2,seconds_3,seconds_4,seconds_5,seconds_6,seconds_7,seconds_8,seconds_9,sum_seconds,mean_seconds
session_id,,,,,,,,,,,,,,,
1,11:19:53,April,Thursday,2014,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,7.0,0.777778
2,11:00:28,March,Friday,2014,25.0,5.0,8.0,3.0,1.0,13.0,6.0,1.0,23.0,85.0,9.444444
3,15:55:12,May,Monday,2014,1.0,1.0,1.0,1.0,1.0,1.0,1.0,14.0,3.0,24.0,2.666667
4,10:03:19,April,Friday,2014,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.444444
5,15:05:31,May,Friday,2014,1.0,1.0,6.0,1.0,0.0,0.0,0.0,0.0,4.0,13.0,1.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82793,18:20:09,February,Monday,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
82794,14:16:40,May,Monday,2014,1.0,3.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,39.0,4.333333
82795,11:21:56,February,Wednesday,2014,0.0,0.0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,0.888889


In [13]:
train_df_times

,first_visit,month,day_name,year,seconds_1,seconds_2,seconds_3,seconds_4,seconds_5,seconds_6,seconds_7,seconds_8,seconds_9,sum_seconds,mean_seconds
session_id,,,,,,,,,,,,,,,
27554,07:39:35,November,Friday,2013,0.0,1.0,0.0,4.0,1.0,9.0,0.0,0.0,35.0,50.0,5.555556
81350,07:44:25,November,Friday,2013,0.0,20.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,24.0,2.666667
234665,07:52:17,November,Friday,2013,1.0,1.0,0.0,1.0,0.0,3.0,3.0,0.0,2.0,11.0,1.222222
97610,07:52:28,November,Friday,2013,1.0,8.0,0.0,33.0,0.0,46.0,54.0,28.0,8.0,178.0,19.777778
161358,07:57:50,November,Friday,2013,1.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,11.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129606,18:06:13,May,Saturday,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
53628,18:09:26,May,Saturday,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
84754,18:11:21,May,Saturday,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [14]:
train_df_sites = train_df[sites].fillna(0)
train_df_sites

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
27554,41475,6725,6725,41475,41476,41476,6725,41475,41476,6725
81350,41476,41475,41476,6725,41475,41476,41476,6725,41475,41476
234665,4802,23,4803,38,38,4804,4804,21,23,22
97610,23,23,22,21,63,66,784,4804,4804,784
161358,41476,41476,6725,41475,41476,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
129606,6003,0,0,0,0,0,0,0,0,0
53628,6003,0,0,0,0,0,0,0,0,0
84754,6003,0,0,0,0,0,0,0,0,0


In [15]:
new_dict = {}
for key in site_dict:
    new_dict[site_dict[key]] = key

In [16]:
for i in range(1, 11):
    train_df_sites[f'site{i}'] = train_df_sites[f'site{i}'].map(new_dict)
train_df_sites

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
27554,security.debian.org,www-fourier.ujf-grenoble.fr,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr
81350,backports.debian.org,security.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org
234665,cnfg.toolbarservices.com,www.google.com,utils.delta-search.com,ajax.googleapis.com,ajax.googleapis.com,img.babylon.com,img.babylon.com,www.google.fr,www.google.com,apis.google.com
97610,www.google.com,www.google.com,apis.google.com,www.google.fr,ieonline.microsoft.com,go.microsoft.com,javadl-esd-secure.oracle.com,img.babylon.com,img.babylon.com,javadl-esd-secure.oracle.com
161358,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
129606,ent-shasta-rrs.symantec.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53628,ent-shasta-rrs.symantec.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84754,ent-shasta-rrs.symantec.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
test_df_sites = test_df[sites].fillna(0)
for i in range(1, 11):
    test_df_sites[f'site{i}'] = test_df_sites[f'site{i}'].map(new_dict)
test_df_sites

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,www.facebook.com,s-static.ak.facebook.com,apis.google.com,c1.adform.net,www.google.com,mpp2.vindicosuite.com,api.nanigans.com,www.google.fr,secure.vistaprint.fr,www.vistaprint.fr
2,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org
3,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,o2i.univ-bpclermont.fr,o2i.univ-bpclermont.fr
4,fr.yahoo.com,yahoo.fr,ocsp.verisign.com,ocsp.thawte.com,rapidssl-ocsp.geotrust.com,clicks.beap.bc.yahoo.com,ocsp.verisign.com,ocsp.digicert.com,ocsp.digicert.com,clicks.beap.bc.yahoo.com
5,sci.sciences.univ-bpclermont.fr,sci.sciences.univ-bpclermont.fr,sci.sciences.univ-bpclermont.fr,go.microsoft.com,windows.microsoft.com,res2.windows.microsoft.com,res1.windows.microsoft.com,ajax.microsoft.com,js.microsoft.com,www.bing.com
...,...,...,...,...,...,...,...,...,...,...
82793,mail.google.com,talkgadget.google.com,accounts.youtube.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82794,www.univ-bpclermont.fr,ent.univ-bpclermont.fr,ent.univ-bpclermont.fr,www.univ-bpclermont.fr,www.univ-bpclermont.fr,reunions.univ-bpclermont.fr,ent.univ-bpclermont.fr,filez.univ-bpclermont.fr,graal.univ-bpclermont.fr,dokudsi.univ-bpclermont.fr
82795,www.facebook.com,static.ak.facebook.com,s-static.ak.facebook.com,apis.google.com,twitter.com,www.rolandgarros.com,platform.twitter.com,www.google.fr,www.google.com,metrics.rolandgarros.com


In [18]:
train_df_join = train_df_sites.fillna(0).join(train_df_times)

train_df_join.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,seconds_2,seconds_3,seconds_4,seconds_5,seconds_6,seconds_7,seconds_8,seconds_9,sum_seconds,mean_seconds
session_id,,,,,,,,,,,,,,,,,,,,,
27554,security.debian.org,www-fourier.ujf-grenoble.fr,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,...,1.0,0.0,4.0,1.0,9.0,0.0,0.0,35.0,50.0,5.555556
81350,backports.debian.org,security.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,...,20.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,24.0,2.666667
234665,cnfg.toolbarservices.com,www.google.com,utils.delta-search.com,ajax.googleapis.com,ajax.googleapis.com,img.babylon.com,img.babylon.com,www.google.fr,www.google.com,apis.google.com,...,1.0,0.0,1.0,0.0,3.0,3.0,0.0,2.0,11.0,1.222222
97610,www.google.com,www.google.com,apis.google.com,www.google.fr,ieonline.microsoft.com,go.microsoft.com,javadl-esd-secure.oracle.com,img.babylon.com,img.babylon.com,javadl-esd-secure.oracle.com,...,8.0,0.0,33.0,0.0,46.0,54.0,28.0,8.0,178.0,19.777778
161358,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,0,0,0,0,0,...,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,11.000000


In [19]:
test_df_join = test_df_sites.fillna(0).join(test_df_times)

test_df_join.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,seconds_2,seconds_3,seconds_4,seconds_5,seconds_6,seconds_7,seconds_8,seconds_9,sum_seconds,mean_seconds
session_id,,,,,,,,,,,,,,,,,,,,,
1,www.facebook.com,s-static.ak.facebook.com,apis.google.com,c1.adform.net,www.google.com,mpp2.vindicosuite.com,api.nanigans.com,www.google.fr,secure.vistaprint.fr,www.vistaprint.fr,...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,7.0,0.777778
2,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,...,5.0,8.0,3.0,1.0,13.0,6.0,1.0,23.0,85.0,9.444444
3,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,o2i.univ-bpclermont.fr,o2i.univ-bpclermont.fr,...,1.0,1.0,1.0,1.0,1.0,1.0,14.0,3.0,24.0,2.666667
4,fr.yahoo.com,yahoo.fr,ocsp.verisign.com,ocsp.thawte.com,rapidssl-ocsp.geotrust.com,clicks.beap.bc.yahoo.com,ocsp.verisign.com,ocsp.digicert.com,ocsp.digicert.com,clicks.beap.bc.yahoo.com,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.444444
5,sci.sciences.univ-bpclermont.fr,sci.sciences.univ-bpclermont.fr,sci.sciences.univ-bpclermont.fr,go.microsoft.com,windows.microsoft.com,res2.windows.microsoft.com,res1.windows.microsoft.com,ajax.microsoft.com,js.microsoft.com,www.bing.com,...,1.0,6.0,1.0,0.0,0.0,0.0,0.0,4.0,13.0,1.444444


For the very basic model, we will use only the visited websites in the session (we will not take into account timestamp features). 

*Alice has her favorite sites, and the more often you see these sites in the session, the higher probability that this is an Alice session, and vice versa.*

Let us prepare the data, we will take only features `site1, site2, ... , site10` from the whole dataframe. Keep in mind that the missing values are replaced with zero. Here is what the first rows of the dataframe look like:

Since IDs of sites have no meaning (does not matter if a site has an ID of 1 or 100), we need to think about how to encode the meaning of "this site in a session means higher probablity that it is an Alice session". 

We will use a technique called ["bag of words plus n-gram model"](https://en.wikipedia.org/wiki/Bag-of-words_model).

We will make a "site-session" matrix analogous to the term-document matrix.

We are not the first, and luckily there is a function CountVectorizer that will implement the above model. Type help(CountVectorizer) to learn about the function. 

We will now initialize a "cv" (CountVectorizer's) instance which we need to train. 

We will use the following parameters:

_ngram range=(1, 3)_ - here we decide that we will use 
1) the name of the site, 
2) two consecutive site names, and 
3) three consecutive site names as features. 
E.g. "google.com" or "google.com vk.com" or "google.com vk.com groups.live.com". 

CountVectorizer will create a large dictionary of 1, 2, and 3-gram strings of sites represented by their numerical IDs. However, this dictionary will be so so large that we may run into trouble with memory or we will just be inefficent chasing phantom combinations.

We will thus limit the dictionary to 50K of the most frequent n-grams:

_max features=50000_

Here is our empty instance:

In [20]:
cv = CountVectorizer(ngram_range=(1, 2), max_features=100000)

CountVectorizer accepts "document strings", so let's prepare a string of our "documents" (i.e. sites), divided by space. Since the string will be huge, we will write this string in a text file using pandas:

In [21]:
train_df_join.fillna(0).to_csv('train_sessions_text.txt', 
                                 sep=' ', index=None, header=None)
test_df_join.fillna(0).to_csv('test_sessions_text.txt', 
                                sep=' ', index=None, header=None)

Before we start using CountVectorizer, let's see how it works on a sub-set of 5 sessions:

In [22]:
five_sess_train = pd.read_csv('train_sessions_text.txt', sep=' ', nrows=5, header=None)
five_sess_test = pd.read_csv('test_sessions_text.txt', sep=' ', nrows=5, header=None)

In [23]:
five_sess_train

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,security.debian.org,www-fourier.ujf-grenoble.fr,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,...,1.0,0.0,4.0,1.0,9.0,0.0,0.0,35.0,50.0,5.555556
1,backports.debian.org,security.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,...,20.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,24.0,2.666667
2,cnfg.toolbarservices.com,www.google.com,utils.delta-search.com,ajax.googleapis.com,ajax.googleapis.com,img.babylon.com,img.babylon.com,www.google.fr,www.google.com,apis.google.com,...,1.0,0.0,1.0,0.0,3.0,3.0,0.0,2.0,11.0,1.222222
3,www.google.com,www.google.com,apis.google.com,www.google.fr,ieonline.microsoft.com,go.microsoft.com,javadl-esd-secure.oracle.com,img.babylon.com,img.babylon.com,javadl-esd-secure.oracle.com,...,8.0,0.0,33.0,0.0,46.0,54.0,28.0,8.0,178.0,19.777778
4,backports.debian.org,backports.debian.org,www-fourier.ujf-grenoble.fr,security.debian.org,backports.debian.org,0,0,0,0,0,...,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,11.000000


First of all, let's make an inverse dictionary which gives us a site name for ID.
The direct dictionary came to us like this:

In [24]:
five_sess_test

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,www.facebook.com,s-static.ak.facebook.com,apis.google.com,c1.adform.net,www.google.com,mpp2.vindicosuite.com,api.nanigans.com,www.google.fr,secure.vistaprint.fr,www.vistaprint.fr,...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,7.0,0.777778
1,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,annotathon.org,...,5.0,8.0,3.0,1.0,13.0,6.0,1.0,23.0,85.0,9.444444
2,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,safebrowsing-cache.google.com,o2i.univ-bpclermont.fr,o2i.univ-bpclermont.fr,...,1.0,1.0,1.0,1.0,1.0,1.0,14.0,3.0,24.0,2.666667
3,fr.yahoo.com,yahoo.fr,ocsp.verisign.com,ocsp.thawte.com,rapidssl-ocsp.geotrust.com,clicks.beap.bc.yahoo.com,ocsp.verisign.com,ocsp.digicert.com,ocsp.digicert.com,clicks.beap.bc.yahoo.com,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.444444
4,sci.sciences.univ-bpclermont.fr,sci.sciences.univ-bpclermont.fr,sci.sciences.univ-bpclermont.fr,go.microsoft.com,windows.microsoft.com,res2.windows.microsoft.com,res1.windows.microsoft.com,ajax.microsoft.com,js.microsoft.com,www.bing.com,...,1.0,6.0,1.0,0.0,0.0,0.0,0.0,4.0,13.0,1.444444


Here is what the fit and transform method -- i.e. learn the dictionary and make the matrix -- produces in our "cv":
a sparse matrix. Why sparse? Because nrows * dict_size = usually will not fit in memory 
(obviously, our 5 sessions will fit in memory so that we can look at them)

Let's see how the first site of the first session, "security.debian.org", is recorded in the session-site matrix. 
Its ID is 21 which corresponds to 3. It is the number of times this site was seen in the first session.
Indeed, count for yourself in the cell above. 

Let's go back to all sessions.

Fit `CountVectorizer` to train data and transform the train and test data with it.

In [25]:
%%time

with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)

print(X_train.shape, X_test.shape)

# Note very big dimensions of matrices: 253561 * 50000 = 12678050000 elements in train! Only sparse matrices can take it.

(253561, 100000) (82797, 100000)
CPU times: total: 23.4 s
Wall time: 23.5 s


In [26]:
X_train

<253561x100000 sparse matrix of type '<class 'numpy.int64'>'
	with 11015887 stored elements in Compressed Sparse Row format>

In [27]:
y_train = train_df['target']
y_train

session_id
27554     0
81350     0
234665    0
97610     0
161358    0
         ..
129606    0
53628     0
84754     0
153398    0
11690     0
Name: target, Length: 253561, dtype: int64

### Training the first model

So, we have an algorithm and data for it. Let us build our first model, using [logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) implementation from ` Sklearn` with default parameters. We will use the first 90% of the data for training (the training data set is sorted by time) and the remaining 10% for validation. Let's write a simple function that returns the quality of the model and then train our first classifier:

In [28]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, solver='lbfgs', max_iter=1000).fit(X[:idx, :], y[:idx])
    print('score=', lr.score(X, y))
    # Prediction for validation set
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    
    # Calculate the quality
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [29]:
# Our target variable
y_train = train_df['target'].values

In [30]:
%%time
# Calculate metric on the validation set. 90% of train data for training. 10% for validation.

get_auc_lr_valid(X_train, y_train, C=1)

score= 0.9979058293665035
CPU times: total: 58.8 s
Wall time: 58.3 s


0.9854461339845126

In [31]:
# 50% of train data for training:

get_auc_lr_valid(X_train, y_train, ratio=0.5)

score= 0.9958668722713666


0.9443478882103324

In [32]:
# Wow! Big data rules in this task: .82 -> .91

In [33]:
# Function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = range(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [34]:
# Train the model on the whole training data set
# Use random_state=17 for reproducibility
# Parameter C=1 by default, but here we set it explicitly

lr = LogisticRegression(C=1, random_state=17, solver='lbfgs', max_iter=1000).fit(X_train, y_train)

# Make a prediction for test data set
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
write_to_submission_file(y_test, 'my_baseline_v1.csv')